In [263]:
import sys
sys.path.append('..')
from db1 import *
import pandas as pd

R = PolynomialRing(ZZ, 't')
t = R.gen()

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
    """
    if n <= 8 and M.is_connected():
        for i in range(len(mat[n][k])):
            if mat[n][k][i].is_isomorphic(M):
                return ikl[n][k][i]
    """
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * kazhdan_lusztig_uniform_matroid(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_uniform_matroid(k, n) + kazhdan_lusztig_uniform_matroid(k-c+1, h) * kazhdan_lusztig_uniform_matroid(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

In [264]:
#creating parallel connection of cycles
m, n = 4, 5

def parallel_connection(m, n):
    G = graphs.CycleGraph(m + n - 2)
    G.add_edge(0, m-1)
    edge_e = frozenset({(0, m - 1)})
    return G, edge_e

In [265]:
S = PolynomialRing(ZZ, 't')
for n in range(3, 13):
    G, e = parallel_connection(n, n)
    Qdiff = S(kl_inverse_fast(Matroid(G)) - uniformQpoly(2*n-3, 2*n-2))
    Qcon = S(uniformQpoly(n-2, n-1))
    print(f"n = {n}, Qdiff = {Qdiff.factor()}")
    print(f"Qcon = {Qcon.factor()}")
    if n % 2 == 0: print("\n")
    if n % 2 != 0:
        quo, rem = Qdiff.quo_rem(Qcon)
        print(f"({S(t+1)}) * Qcon = {S(t+1) * Qcon}")
        print(f"{quo.factor()} = ({(S(t+1) * Qcon)}) + {quo - S(t+1) * Qcon}\n")


n = 3, Qdiff = (-1) * (t - 1)
Qcon = 1
(t + 1) * Qcon = t + 1
(-1) * (t - 1) = (t + 1) + -2*t

n = 4, Qdiff = 2^2 * (t + 1)
Qcon = 2


n = 5, Qdiff = (-1) * (t - 3) * (2*t + 1) * (2*t + 3)
Qcon = 2*t + 3
(t + 1) * Qcon = 2*t^2 + 5*t + 3
(-1) * (t - 3) * (2*t + 1) = (2*t^2 + 5*t + 3) + -4*t^2

n = 6, Qdiff = (t + 1) * (5*t + 4)^2
Qcon = 5*t + 4


n = 7, Qdiff = (-1) * (5*t^2 + 9*t + 5) * (5*t^3 - 14*t^2 - 14*t - 5)
Qcon = 5*t^2 + 9*t + 5
(t + 1) * Qcon = 5*t^3 + 14*t^2 + 14*t + 5
(-1) * (5*t^3 - 14*t^2 - 14*t - 5) = (5*t^3 + 14*t^2 + 14*t + 5) + -10*t^3

n = 8, Qdiff = 2^2 * (t + 1) * (7*t^2 + 7*t + 3)^2
Qcon = 2 * (7*t^2 + 7*t + 3)


n = 9, Qdiff = (-1) * (14*t^3 + 28*t^2 + 20*t + 7) * (14*t^4 - 42*t^3 - 48*t^2 - 27*t - 7)
Qcon = 14*t^3 + 28*t^2 + 20*t + 7
(t + 1) * Qcon = 14*t^4 + 42*t^3 + 48*t^2 + 27*t + 7
(-1) * (14*t^4 - 42*t^3 - 48*t^2 - 27*t - 7) = (14*t^4 + 42*t^3 + 48*t^2 + 27*t + 7) + -28*t^4

n = 10, Qdiff = (t + 1) * (42*t^3 + 48*t^2 + 27*t + 8)^2
Qcon = 42*t^3 + 48*t^2 + 27

In [266]:
for n in range(3, 13):
    Qcon = S(uniformQpoly(n-2, n-1))
    G, e = parallel_connection(n, n)
    Qdiff = S(kl_inverse_fast(Matroid(G)) - uniformQpoly(2*n-3, 2*n-2))
    print(f"n = {n}, Qdiff = {Qdiff}, Qcon = {Qcon}")
    p = S(t+1) * Qcon ** 2
    print(f"{Qdiff} = ({p}) + {(Qdiff - p).factor() if (Qdiff - p) else 0}\n")

n = 3, Qdiff = -t + 1, Qcon = 1
-t + 1 = (t + 1) + (-1) * 2 * t

n = 4, Qdiff = 4*t + 4, Qcon = 2
4*t + 4 = (4*t + 4) + 0

n = 5, Qdiff = -4*t^3 + 4*t^2 + 21*t + 9, Qcon = 2*t + 3
-4*t^3 + 4*t^2 + 21*t + 9 = (4*t^3 + 16*t^2 + 21*t + 9) + (-1) * 2^2 * (2*t + 3) * t^2

n = 6, Qdiff = 25*t^3 + 65*t^2 + 56*t + 16, Qcon = 5*t + 4
25*t^3 + 65*t^2 + 56*t + 16 = (25*t^3 + 65*t^2 + 56*t + 16) + 0

n = 7, Qdiff = -25*t^5 + 25*t^4 + 171*t^3 + 221*t^2 + 115*t + 25, Qcon = 5*t^2 + 9*t + 5
-25*t^5 + 25*t^4 + 171*t^3 + 221*t^2 + 115*t + 25 = (25*t^5 + 115*t^4 + 221*t^3 + 221*t^2 + 115*t + 25) + (-1) * 2 * 5 * t^3 * (5*t^2 + 9*t + 5)

n = 8, Qdiff = 196*t^5 + 588*t^4 + 756*t^3 + 532*t^2 + 204*t + 36, Qcon = 14*t^2 + 14*t + 6
196*t^5 + 588*t^4 + 756*t^3 + 532*t^2 + 204*t + 36 = (196*t^5 + 588*t^4 + 756*t^3 + 532*t^2 + 204*t + 36) + 0

n = 9, Qdiff = -196*t^7 + 196*t^6 + 1568*t^5 + 2464*t^4 + 2108*t^3 + 1072*t^2 + 329*t + 49, Qcon = 14*t^3 + 28*t^2 + 20*t + 7
-196*t^7 + 196*t^6 + 1568*t^5 + 2464*t^4 + 2

In [267]:
for n in range(3, 13):
    Qcon = S(uniformQpoly(n-2, n-1))
    p = S(t+1) * Qcon ** 2 - 2 * Qcon * Qcon[(n-3)//2] * t ** ((n-1)//2)
    print(f"n = {n}, p = {p} and lt = {Qcon[(n-3)//2] * t ** ((n-1)//2)}\n")

n = 3, p = -t + 1 and lt = t

n = 4, p = -4*t + 4 and lt = 2*t

n = 5, p = -4*t^3 + 4*t^2 + 21*t + 9 and lt = 2*t^2

n = 6, p = -25*t^3 + 25*t^2 + 56*t + 16 and lt = 5*t^2

n = 7, p = -25*t^5 + 25*t^4 + 171*t^3 + 221*t^2 + 115*t + 25 and lt = 5*t^3

n = 8, p = -196*t^5 + 196*t^4 + 588*t^3 + 532*t^2 + 204*t + 36 and lt = 14*t^3

n = 9, p = -196*t^7 + 196*t^6 + 1568*t^5 + 2464*t^4 + 2108*t^3 + 1072*t^2 + 329*t + 49 and lt = 14*t^4

n = 10, p = -1764*t^7 + 1764*t^6 + 6336*t^5 + 7164*t^4 + 4761*t^3 + 1929*t^2 + 496*t + 64 and lt = 42*t^4

n = 11, p = -1764*t^9 + 1764*t^8 + 15660*t^7 + 27900*t^6 + 28365*t^5 + 19551*t^4 + 9445*t^3 + 3205*t^2 + 711*t + 81 and lt = 42*t^5

n = 12, p = -17424*t^9 + 17424*t^8 + 70785*t^7 + 92565*t^6 + 74536*t^5 + 42240*t^4 + 17116*t^3 + 5016*t^2 + 980*t + 100 and lt = 132*t^5

